# Flair Prediction for r/AmItheAsshole

This notebook works through the application of machine learning classification models to attempt to predict the flairs of posts in r/AmItheAsshole (r/AITA) based on their CountVectorized text content.

Session setup is done below:

In [2]:
# Setup - Run only once per Kernel App
%conda install openjdk -y

# install PySpark
%pip install pyspark==3.4.0

# install spark-nlp
%pip install spark-nlp==5.1.3

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - openjdk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.08.22 |       h06a4308_0         123 KB
    certifi-2023.11.17         |  py310h06a4308_0         158 KB
    openjdk-11.0.13            |       h87a67e3_0       341.0 MB
    ------------------------------------------------------------
                                           Total:       341.3 MB

The following NEW packages will be INSTALLED:

  openjdk            pkgs/main/linux-64::openjdk-11.0.13-h87a67e3_0 



In [2]:
import sagemaker
sess = sagemaker.Session()
bucket = sess.default_bucket()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
import sparknlp
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier#, MultilayerPerceptronClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import re
import os
import json
import random
import pyspark.sql.functions as F
from sparknlp.base import *
from pyspark.ml import Pipeline
from sparknlp.annotator import *
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.functions import col, when
from sklearn.metrics import confusion_matrix

In [4]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","32G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")\
    .config("fs.s3a.aws.credentials.provider", "com.amazonaws.auth.ContainerCredentialsProvider")\
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-dbed4efb-d4b8-41d5-bcd3-25bc98d8a482;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 470ms :: artifacts dl 19ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------

In [5]:
print(f"Spark version: {spark.version}")
print(f"sparknlp version: {sparknlp.version()}")

Spark version: 3.4.0
sparknlp version: 5.1.3


## Reading in the Data

Below, the CountVectorized data of text submissions are read in, then filtered to only posts in r/AITA with one of the 4 primary flairs attached (i.e., what we are trying to predict).

In [6]:
%%time
# Read in data from project bucket
bucket = "project17-bucket-alex"
directory = "matt-submissions-cv"

s3_path = f"s3a://{bucket}/{directory}"
submissions_cv = spark.read.parquet(s3_path, header = True)
# Here we subset the submissions to only include posts from r/AmItheAsshole for the subsequent analysis
raw_aita = submissions_cv.filter(F.col('subreddit') == "AmItheAsshole")

# filter submissions to remove deleted/removed posts
aita = raw_aita.filter((F.col('selftext') != '[removed]') & (F.col('selftext') != '[deleted]' ))

# Filter submissions to only include posts tagged with the 4 primary flairs
acceptable_flairs = ['Everyone Sucks', 'Not the A-hole', 'No A-holes here', 'Asshole']
df_flairs = aita.where(F.col('link_flair_text').isin(acceptable_flairs))
df_flairs.select("subreddit", "author", "title", "selftext", "created_utc", "num_comments", "link_flair_text").show()
print(f"shape of the subsetted submissions dataframe of appropriately flaired posts is {df_flairs.count():,}x{len(df_flairs.columns)}")

23/11/29 15:52:40 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
23/11/29 15:52:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------+--------------------+--------------------+--------------------+-------------------+------------+---------------+
|    subreddit|              author|               title|            selftext|        created_utc|num_comments|link_flair_text|
+-------------+--------------------+--------------------+--------------------+-------------------+------------+---------------+
|AmItheAsshole|         Squeakitout|AITA for being pi...|my boyfriendmand ...|2022-03-18 00:34:50|          24| Everyone Sucks|
|AmItheAsshole| Foreign_Quarter8959|WIBTA if I don't ...|so if have been d...|2022-03-18 00:35:50|          28|No A-holes here|
|AmItheAsshole|         100000nopes|AITA for giving a...|i moved into a qu...|2022-03-18 00:38:16|          19| Not the A-hole|
|AmItheAsshole|      MonkeyBeBoolin|AITA for leaving ...|tldr i live with ...|2022-03-18 00:39:29|         107| Not the A-hole|
|AmItheAsshole|Potential-Persimmon3|AITA for wanting ...|for background i’...|2022-03-18 00:44:07|      

shape of the subsetted submissions dataframe of appropriately flaired posts is 110,386x570
CPU times: user 38 ms, sys: 4.53 ms, total: 42.6 ms
Wall time: 26.3 s


Next, we look at a small sample of the words in the vocabulary that will be used to predict the flairs. 

In [8]:
# extract vocabulary from dataframe
word_cols = [col for col in df_flairs.columns if 'word_' in col]
vocabulary = [word.replace('word_', '') for word in word_cols]

# print the first ten vocabulary words
print(f"First ten vocabulary words: {', '.join(vocabulary[:10])}")

First ten vocabulary words: like, feel, want, know, time, tell, get, im, think, friend


From this from this vocabulary and word columns, we can establish a SparkML pipeline an employ a multi-class classification model to predict the flairs associated with each subreddit. We have an unbalanced dataset where the flair Not the A-hole is overrepresented, so we will generate a balanced sample to account for this to ensure more accurate predictions. This resulting "balanced" dataframe contains about 15% of all properly flaired r/AITA posts.

In [8]:
%%time
min_count = df_flairs.groupBy("link_flair_text").count().agg({"count": "min"}).collect()[0][0]

# Create a dataframe with an equal amount of each category
balanced_df = df_flairs.sampleBy("link_flair_text", fractions={val: min_count / df_flairs.filter(col("link_flair_text") == val).count() for val in ['Asshole', 'No A-holes here', 'Everyone Sucks', 'Not the A-hole']})

# Show the result
balanced_df.select("subreddit", "author", "title", "score", "selftext", "created_utc", "num_comments", "link_flair_text").show()

### OLD CODE ###
# Find weights of classes depending on their prevalence in the original dataset
#class_counts = df_flairs.groupBy("link_flair_text").count().collect()
#total_count = df_flairs.count()
#class_weights = {row["link_flair_text"]: total_count / (row["count"] * len(class_counts)) for row in class_counts}

# Add a new column for class weights
#df_flairs = df_flairs.withColumn("class_weight", when(col('link_flair_text') == 'Not the A-hole', class_weights['Not the A-hole'])
#                                 .when(col('link_flair_text') == 'Everyone Sucks', class_weights['Everyone Sucks'])
#                                 .when(col('link_flair_text') == 'Asshole', class_weights['Asshole'])
#                                 .otherwise(class_weights['No A-holes here']))


# Add a new column to the existing dataframe with class weights
#for label, weight in class_weights.items():
#    df_flairs = df_flairs.withColumn("classWeight", F.when(F.col("link_flair_text") == label, weight).otherwise(F.col("classWeight")))

+-------------+--------------------+--------------------+-----+--------------------+-------------------+------------+---------------+
|    subreddit|              author|               title|score|            selftext|        created_utc|num_comments|link_flair_text|
+-------------+--------------------+--------------------+-----+--------------------+-------------------+------------+---------------+
|AmItheAsshole|         Squeakitout|AITA for being pi...|    2|my boyfriendmand ...|2022-03-18 00:34:50|          24| Everyone Sucks|
|AmItheAsshole| Foreign_Quarter8959|WIBTA if I don't ...|    1|so if have been d...|2022-03-18 00:35:50|          28|No A-holes here|
|AmItheAsshole|     freddyfazzballs|AITA for telling ...|    7|so im going to tr...|2022-03-02 03:25:13|          15|No A-holes here|
|AmItheAsshole|   Gold-Seaweed-7673|AITA for pesterin...|   22|i am a picture/me...|2022-03-12 23:23:46|         134|        Asshole|
|AmItheAsshole|Embarrassed_Push_768|AITA For throwing...|    9

In [11]:
train_data, test_data = balanced_df.randomSplit([0.8, 0.2], 24)

In [19]:
%%time
stringIndexer_flair = StringIndexer(inputCol = "link_flair_text", outputCol = "flair_idx")
subreddit_labels = stringIndexer_flair.fit(balanced_df).labels
print(subreddit_labels)
# create a vector assembler with the appropriate input variables
vectorAssembler_features = VectorAssembler(
    inputCols = word_cols, 
    outputCol = 'input_features')
# create the random forest classification model
model = RandomForestClassifier(
    labelCol = 'flair_idx',
    featuresCol = 'input_features',
    numTrees = 50)
# create a label converter to bring the numeric predictions back to string labels
labelConverter = IndexToString(
    inputCol = 'prediction', 
    outputCol = 'predicted_flair', 
    labels = subreddit_labels)
# create the pipline with appropriate stages
pipeline_model = Pipeline(
    stages = [stringIndexer_flair,
              vectorAssembler_features, 
              model, labelConverter])

['Asshole', 'Everyone Sucks', 'No A-holes here', 'Not the A-hole']
CPU times: user 26.5 ms, sys: 395 µs, total: 26.9 ms
Wall time: 12.4 s


In [20]:
%%time
# fit the model
model = pipeline_model.fit(train_data)
                                                                                
# transform the data by applying the model
train_predictions = model.transform(train_data)
predictions = model.transform(test_data)

CPU times: user 203 ms, sys: 61.9 ms, total: 265 ms
Wall time: 2min 37s


Below are the calculations of metrics for the training data.

In [21]:
%%time
evaluator = MulticlassClassificationEvaluator(labelCol = 'flair_idx',
                                              predictionCol = 'prediction',
                                              metricName = 'accuracy')
train_accuracy = evaluator.evaluate(train_predictions)

evaluator = MulticlassClassificationEvaluator(labelCol = 'flair_idx',
                                              predictionCol = 'prediction',
                                              metricName = 'f1')
train_f1 = evaluator.evaluate(train_predictions)

evaluator = MulticlassClassificationEvaluator(labelCol = 'flair_idx',
                                              predictionCol = 'prediction',
                                              metricName = 'weightedPrecision')
train_precision = evaluator.evaluate(train_predictions)

evaluator = MulticlassClassificationEvaluator(labelCol = 'flair_idx',
                                              predictionCol = 'prediction',
                                              metricName = 'weightedRecall')
train_recall = evaluator.evaluate(train_predictions)

CPU times: user 57.7 ms, sys: 48.7 ms, total: 106 ms
Wall time: 2min 19s


In [22]:
print("Training Accuracy:"+str(train_accuracy))
print("Training F1:"+str(train_f1))
print("Training Weighted Precision:"+str(train_precision))
print("Training Weighted Recall:"+str(train_recall))

Training Accuracy:0.4202260384558931
Training F1:0.3827044971013094
Training Weighted Precision:0.45925773965325956
Training Weighted Recall:0.4202260384558931


With our model constructed and fitted, we can now see how accurate it fit to and classified our testing subset of the r/AITA posts.

In [23]:
%%time
evaluator = MulticlassClassificationEvaluator(labelCol = 'flair_idx',
                                              predictionCol = 'prediction',
                                              metricName = 'accuracy')
test_accuracy = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol = 'flair_idx',
                                              predictionCol = 'prediction',
                                              metricName = 'f1')
test_f1 = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol = 'flair_idx',
                                              predictionCol = 'prediction',
                                              metricName = 'weightedPrecision')
test_precision = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol = 'flair_idx',
                                              predictionCol = 'prediction',
                                              metricName = 'weightedRecall')
test_recall = evaluator.evaluate(predictions)

CPU times: user 70 ms, sys: 32.8 ms, total: 103 ms
Wall time: 2min 23s


In [24]:
print("Testing Accuracy:"+str(test_accuracy))
print("Testing F1:"+str(test_f1))
print("Testing Weighted Precision:"+str(test_precision))
print("Testing Weighted Recall:"+str(test_recall))

Testing Accuracy:0.32919254658385094
Testing F1:0.2811499269831472
Testing Weighted Precision:0.3094235533580229
Testing Weighted Recall:0.3291925465838509


In [27]:
# Collect data for confusion matrix plot
train_flair_pred = train_predictions.select("predicted_flair").collect()
train_flair_orig = train_predictions.select("link_flair_text").collect()                                           
flair_pred = predictions.select("predicted_flair").collect()
flair_orig = predictions.select("link_flair_text").collect()      

In [29]:
cm_labels = ['Asshole', 'Everyone Sucks', 'No A-holes here', 'Not the A-hole']
train_cm = confusion_matrix(train_flair_orig, train_flair_pred)
print("Training Confusion Matrix:")
print(pd.DataFrame(train_cm, columns = cm_labels, index = cm_labels))

Training Confusion Matrix:
                 Asshole  Everyone Sucks  No A-holes here  Not the A-hole
Asshole             1531             998              508             311
Everyone Sucks       959            1977              408              81
No A-holes here     2309             649              401              69
Not the A-hole      1281             924             1129              91


In [30]:
cm_labels = ['Asshole', 'Everyone Sucks', 'No A-holes here', 'Not the A-hole']
test_cm = confusion_matrix(flair_orig, flair_pred)
print("Testing Confusion Matrix:")
print(pd.DataFrame(test_cm, columns = cm_labels, index = cm_labels))

Testing Confusion Matrix:
                 Asshole  Everyone Sucks  No A-holes here  Not the A-hole
Asshole              416             289              168              29
Everyone Sucks       238             388              150              36
No A-holes here      494             165              108              22
Not the A-hole       346             302              202              28


In [31]:
# Save confusion mx data to repo  
pd.DataFrame(train_cm, columns = cm_labels, index = cm_labels).to_csv('../../data/ml-data/flair-text-cm-train.csv')
pd.DataFrame(test_cm, columns = cm_labels, index = cm_labels).to_csv('../../data/ml-data/flair-text-cm-test.csv')
# Save model metrics to dataframe
model_metrics = {"training": [train_accuracy, train_f1, train_precision, train_recall],
                "test": [test_accuracy, test_f1, test_precision, test_recall]}
metric_df = pd.DataFrame(model_metrics, index = ['accuracy', 'f1', 'precision', 'recall'])
metric_df.to_csv('../../data/ml-data/flair-text-model-metrics-df.csv')

From the above metrics and confusion matrix, we can see that this Random Forest model does not do a very good job at predicting the flair based on the text content of the posts when accounting for the frequency of the classes, but it still performs better than performing a purely random selection (in which case the accuracy would be 0.25). Based on the confusion matrices, we can see that the model does not predict "Not the A-hole" posts very well, and in turn struggles to differentiate between posts that are rated as "Asshole" and "No A-holes here". However, the model performs relatively better when predicting the flair "Everyone Sucks" for both the training and testing data sets. While this model is slightly better than pure random assignment of flairs, it is still not a very effective method at predicting the Reddit judgment of these posts (i.e., what flair is assigned). Visualizations related to this section are generated in a separate notebook.

Below, we attempt another classification model, but instead using number of comments and post score as predictors. The order of steps is identical to the text based predictions above, but we instead look at user engagement.

In [9]:
%%time
stringIndexer_flair = StringIndexer(inputCol = "link_flair_text", outputCol = "flair_idx")
subreddit_labels = stringIndexer_flair.fit(balanced_df).labels
print(subreddit_labels)
# create a vector assembler with the appropriate input variables
vectorAssembler_features = VectorAssembler(
    inputCols = ['num_comments', 'score'], 
    outputCol = 'input_features')
# create the random forest classification model
model = RandomForestClassifier(
    labelCol = 'flair_idx',
    featuresCol = 'input_features',
    numTrees = 50)
# create a label converter to bring the numeric predictions back to string labels
labelConverter = IndexToString(
    inputCol = 'prediction', 
    outputCol = 'predicted_flair', 
    labels = subreddit_labels)
# create the pipline with appropriate stages
pipeline_model = Pipeline(
    stages = [stringIndexer_flair,
              vectorAssembler_features, 
              model, labelConverter])

['No A-holes here', 'Not the A-hole', 'Everyone Sucks', 'Asshole']
CPU times: user 21 ms, sys: 5.8 ms, total: 26.8 ms
Wall time: 13.1 s


In [12]:
%%time
# fit the model
model = pipeline_model.fit(train_data)
                                                                                
# transform the data by applying the model
train_predictions = model.transform(train_data)
predictions = model.transform(test_data)

CPU times: user 129 ms, sys: 31.3 ms, total: 161 ms
Wall time: 2min 38s


In [13]:
%%time
evaluator = MulticlassClassificationEvaluator(labelCol = 'flair_idx',
                                              predictionCol = 'prediction',
                                              metricName = 'accuracy')
train_accuracy = evaluator.evaluate(train_predictions)

evaluator = MulticlassClassificationEvaluator(labelCol = 'flair_idx',
                                              predictionCol = 'prediction',
                                              metricName = 'f1')
train_f1 = evaluator.evaluate(train_predictions)

evaluator = MulticlassClassificationEvaluator(labelCol = 'flair_idx',
                                              predictionCol = 'prediction',
                                              metricName = 'weightedPrecision')
train_precision = evaluator.evaluate(train_predictions)

evaluator = MulticlassClassificationEvaluator(labelCol = 'flair_idx',
                                              predictionCol = 'prediction',
                                              metricName = 'weightedRecall')
train_recall = evaluator.evaluate(train_predictions)

CPU times: user 78.6 ms, sys: 31.7 ms, total: 110 ms
Wall time: 2min 25s


In [14]:
print("Training Accuracy:"+str(train_accuracy))
print("Training F1:"+str(train_f1))
print("Training Weighted Precision:"+str(train_precision))
print("Training Weighted Recall:"+str(train_recall))

Training Accuracy:0.3963784183296378
Training F1:0.39380938939184995
Training Weighted Precision:0.39584127929986934
Training Weighted Recall:0.3963784183296378


In [15]:
%%time
evaluator = MulticlassClassificationEvaluator(labelCol = 'flair_idx',
                                              predictionCol = 'prediction',
                                              metricName = 'accuracy')
test_accuracy = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol = 'flair_idx',
                                              predictionCol = 'prediction',
                                              metricName = 'f1')
test_f1 = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol = 'flair_idx',
                                              predictionCol = 'prediction',
                                              metricName = 'weightedPrecision')
test_precision = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol = 'flair_idx',
                                              predictionCol = 'prediction',
                                              metricName = 'weightedRecall')
test_recall = evaluator.evaluate(predictions)

CPU times: user 72.1 ms, sys: 35.3 ms, total: 107 ms
Wall time: 2min 22s


In [16]:
print("Testing Accuracy:"+str(test_accuracy))
print("Testing F1:"+str(test_f1))
print("Testing Weighted Precision:"+str(test_precision))
print("Testing Weighted Recall:"+str(test_recall))

Testing Accuracy:0.39613670133729567
Testing F1:0.392595251057266
Testing Weighted Precision:0.3948058303170742
Testing Weighted Recall:0.3961367013372957


In [17]:
# Collect data for confusion matrix plot
train_flair_pred = train_predictions.select("predicted_flair").collect()
train_flair_orig = train_predictions.select("link_flair_text").collect()                                           
flair_pred = predictions.select("predicted_flair").collect()
flair_orig = predictions.select("link_flair_text").collect()      

In [18]:
cm_labels = ['Asshole', 'Everyone Sucks', 'No A-holes here', 'Not the A-hole']
train_cm = confusion_matrix(train_flair_orig, train_flair_pred)
print("Training Confusion Matrix:")
print(pd.DataFrame(train_cm, columns = cm_labels, index = cm_labels))

Training Confusion Matrix:
                 Asshole  Everyone Sucks  No A-holes here  Not the A-hole
Asshole             1609             705              468             570
Everyone Sucks       833            1079              617             865
No A-holes here      619            1261              802             696
Not the A-hole       393             984             1628             401


In [19]:
cm_labels = ['Asshole', 'Everyone Sucks', 'No A-holes here', 'Not the A-hole']
test_cm = confusion_matrix(flair_orig, flair_pred)
print("Testing Confusion Matrix:")
print(pd.DataFrame(test_cm, columns = cm_labels, index = cm_labels))

Testing Confusion Matrix:
                 Asshole  Everyone Sucks  No A-holes here  Not the A-hole
Asshole              406             182              101             150
Everyone Sucks       191             280              175             197
No A-holes here      153             317              206             167
Not the A-hole        91             247              413              89


In [20]:
# Save confusion mx data to repo  
pd.DataFrame(train_cm, columns = cm_labels, index = cm_labels).to_csv('../../data/ml-data/flair-engagement-cm-train.csv')
pd.DataFrame(test_cm, columns = cm_labels, index = cm_labels).to_csv('../../data/ml-data/flair-engagement-cm-test.csv')
# Save model metrics to dataframe
model_metrics = {"training": [train_accuracy, train_f1, train_precision, train_recall],
                "test": [test_accuracy, test_f1, test_precision, test_recall]}
metric_df = pd.DataFrame(model_metrics, index = ['accuracy', 'f1', 'precision', 'recall'])
metric_df.to_csv('../../data/ml-data/flair-engagement-model-metrics-df.csv')

Based on the model metrics and confusion matrices above, we can see that this model performs slightly better than the flair predictions based on the text content of the posts given the ~40% accuracy for both the training and test data sets. All of the model metrics for the engagement based model exceed that of the CountVectorizer/word based model. This engagement based model continues to struggle to correctly identify Not the A-hole posts, but performs reasonably well at predicting Asshole flaired posts, identifying them correctly the majority of the time. While this model still is not extremely useful given its suboptimal accuracy, it is better performing than a random guess and a word/text-based model. Visualizations of these model metrics and confusion matrices are performed in a separate notebook.